In [6]:
%%capture
!pip install langchain_core langchain_core
!pip install langchain_openai langgraph
!pip install grandalf

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
#from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

generation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a twitter techie influencer assistant tasked with writing excellent twitter posts."
            " Generate the best twitter post possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a viral twitter influencer grading a tweet. Generate critique and recommendations for the user's tweet."
            "Always provide detailed recommendations, including requests for length, virality, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
from google.colab import userdata
llm = ChatOpenAI(model="gpt-4o",api_key=userdata.get('lp'))
generation_chain = generation_prompt | llm
reflection_chain = reflection_prompt | llm

In [8]:
from typing import List, Sequence
#from dotenv import load_dotenv
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, MessageGraph
#from chains import generation_chain, reflection_chain

#load_dotenv()

REFLECT = "reflect"
GENERATE = "generate"
graph = MessageGraph()

def generate_node(state):
    return generation_chain.invoke({
        "messages": state
    })


def reflect_node(messages):
    response = reflection_chain.invoke({
        "messages": messages
    })
    return [HumanMessage(content=response.content)]


graph.add_node(GENERATE, generate_node)
graph.add_node(REFLECT, reflect_node)
graph.set_entry_point(GENERATE)


def should_continue(state):
    if (len(state) > 6):
        return END
    return REFLECT


graph.add_conditional_edges(GENERATE, should_continue)
graph.add_edge(REFLECT, GENERATE)

app = graph.compile()

print(app.get_graph().draw_mermaid())
app.get_graph().print_ascii()

response = app.invoke(HumanMessage(content="AI Agents taking over content creation"))

print(response)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__(<p>__start__</p>)
	generate(generate)
	reflect(reflect)
	__end__(<p>__end__</p>)
	__start__ --> generate;
	generate --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
+----------+   
| generate |   
+----------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   
[HumanMessage(content='AI Agents taking over content creation', additional_kwargs={}, response_metadata={}, id='5a485053-94b0-471c-ad64-335ce0b4a8c8'), AIMessage(content='🌟 The future of creativity is here! AI agents are revolutionizing content creation, breaking the barriers of imagination and efficiency. 🚀 From drafting articles to generating stunning visuals, the possibilities are endless. Are you ready to collaborate with AI and take your creativit

In [10]:
print(response[-1].content)

🚀 Are AI agents just assistants in content creation, or are they rewriting the rulebook? From drafting engaging articles to designing eye-catching visuals, their potential is thrilling! Imagine letting AI craft your next viral sensation—how great would that be? Share your thoughts or experiences! 📈🤖✨ What role do you think they play today, and what’s possible for tomorrow? #AI #ContentCreation #DigitalInnovation #FutureTrends


In [11]:
response[-1].content

'🚀 Are AI agents just assistants in content creation, or are they rewriting the rulebook? From drafting engaging articles to designing eye-catching visuals, their potential is thrilling! Imagine letting AI craft your next viral sensation—how great would that be? Share your thoughts or experiences! 📈🤖✨ What role do you think they play today, and what’s possible for tomorrow? #AI #ContentCreation #DigitalInnovation #FutureTrends'

In [12]:
response = app.invoke(HumanMessage(content=" search the job on datascince"))
response[-1].content

"Dreaming of a career in data science? 🚀 Here’s your roadmap to success: \n\n1️⃣ **Master the Essentials:** Dive into Python, R, SQL & data visualization.  \n2️⃣ **Showcase Your Talent:** Use GitHub & Kaggle to highlight your amazing projects.  \n3️⃣ **Network Effectively:** Build connections on LinkedIn & attend industry events.  \n4️⃣ **Learn Continuously:** Stay ahead with top blogs and online courses.  \n5️⃣ **Ace the Interview:** Hone problem-solving & data storytelling skills.\n\nData science isn't just a job—it's an adventure of discovery, like unraveling mysteries with data! What's your journey been like? Share your story or tag someone who could use these tips! 🔍✨ #DataScience #CareerGrowth #DreamBig\n\nDon't forget to pair this tweet with an inspiring image or infographic for more visual impact! 🌟"